# Summary
The NeuroGeriatric Motion Toolbox (NGMT) is an open-source Python toolbox designed for processing human motion data. Targeting motion researchers and clinicians, NGMT provides a comprehensive suite of algorithms for motion data processing in neuroscience and biomechanics, including gait sequence detection, initial contact detection, pyhisical activity monitoring, etc. These algorithms assist in assessing human motion data and identifying abnormalities, facilitating diagnosis and treatment planning for individuals with mobility impairments. The modular design of NGMT allows for flexibility and scalability, making it suitable for a wide range of applications in human motion analysis research.

# Statement of need
The analysis of human movement is critical for neurological assessment [@micoamigo_2023]. Human motion characterization is crucial for overall well-being, encompassing our physical, mental, and social dimensions. The increasing prevalence of mobility-limiting diseases, such as Parkinson's disease (PD), poses a serious burden on healthcare systems [mahlknecht2013prevalence]. Wearable devices such as inertial measurement units (IMUs) allow for long-term monitoring of disease progression and could, therefore, be used to track changes in gait [mazza2021technical]. While many studies have focused on a single IMU worn on the lower back, they often used not freely available software to extract data [mico2023assessing]. The development of easy-to-use and open-source code implementations is imperative for large-scale data extraction in research and clinical settings. NGMT addresses this gap by providing diverse algorithms for human motion data analysis, catering to motion researchers and clinicians and promoting the utilization of open-source software. NGMT encompasses a broad range of validated algorithms such as gait sequence detection, initial contact detection, and physical activity monitoring. While not exhaustive, ongoing efforts aim to include additional validated algorithms, such as sit-to-stand, stand-to-sit, etc.

# Provided Functionality
NGMT offers practical examples demonstrating the application of currently implemented algorithms, focusing on gait sequence detection and initial contact detection. The toolbox utilizes IMU sensor data from clinical cohorts, such as those with congestive heart failure (CHF) [@mico2023assessing]. Participants undergo real-world assessments, engaging in daily activities and specific tasks, including outdoor walking, navigating slopes and stairs, and moving between rooms. 

The data were processed using our gait sequence detection module, based on the Paraschiv-Ionescu algorithm[paraschiv2019locomotion,paraschiv2020real], to identify gait sequences within the time series. Subsequently, our initial contact detection module , also based on the Paraschiv-Ionescu algorithm [paraschiv2019locomotion,paraschiv2020real], was applied to identify initial contacts within the detected gait sequences. Utilizing lower back acceleration data from the Mobilise-D dataset, we demonstrate the accurate identification of gait events such as gait onset, gait duration, and initial contacts. NGMT offers practical examples demonstrating the application of currently implemented algorithms, focusing on gait sequence detection and initial contact detection. This example illustrates the practical application of our toolbox in analyzing human gait patterns which is shown in figure 1.

In figure 1, we present a detailed analysis of acceleration data from the lower back, highlighting the key gait events detected by our modules. The green vertical line indicates the onset of a gait sequence, while the shaded gray region represents the duration of the gait sequence. Additionally, blue dashed lines denote the detected initial contacts within the gait sequence.

# Installation and usage
The NGMT package is implemented in Python and is freely available under a Non-Profit Open Software License version 3.0. The stable version of the package can be installed from PyPI.org using `pip install ngmt`. Our documentation provides detailed instructions on installation and some tutorial notebooks.

# Acknowledgements

# References

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ngmt.datasets import mobilised
from ngmt.modules.gsd import ParaschivIonescuGaitSequenceDetection
from ngmt.modules.icd import ParaschivIonescuInitialContactDetection
from ngmt.config import cfg_colors

# Read the data
file_path = (
    r"C:\Users\Project\Desktop\Gait_Sequence\Mobilise-D dataset_1-18-2023\CHF\data.mat"
)

# Define tracking system and tracked points
tracking_sys = "SU"
tracked_points = {tracking_sys: ["LowerBack"]}

# Load recording data
recording = mobilised.load_recording(
    file_name=file_path, tracking_systems=[tracking_sys], tracked_points=tracked_points
)

# Extract lower back acceleration data
acceleration_data = recording.data[tracking_sys][
    ["LowerBack_ACCEL_x", "LowerBack_ACCEL_y", "LowerBack_ACCEL_z"]
]

# Get sampling frequency
sampling_frequency = recording.channels[tracking_sys][
    recording.channels[tracking_sys]["name"] == "LowerBack_ACCEL_x"
]["sampling_frequency"].values[0]

# Gait sequence detection
gsd = ParaschivIonescuGaitSequenceDetection(target_sampling_freq_Hz=40)
gsd = gsd.detect(
    data=acceleration_data, sampling_freq_Hz=sampling_frequency, plot_results=False
)
gait_sequences = gsd.gait_sequences_

# Initial contact detection
icd = ParaschivIonescuInitialContactDetection(target_sampling_freq_Hz=40)
icd = icd.detect(
    data=acceleration_data,
    gait_sequences=gait_sequences,
    sampling_freq_Hz=sampling_frequency,
)
initial_contacts = icd.initial_contacts_

# Access the first detected gait sequence
first_gait_sequence = gait_sequences.iloc[0]

# Plot setup
fig, ax = plt.subplots(figsize=(12, 8))
colors = cfg_colors["raw"]
num_samples = len(acceleration_data)
time_seconds = np.arange(num_samples) / sampling_frequency

# Initial contacts within the first gait sequence
ic_within_gait = initial_contacts[
    initial_contacts["onset"].between(
        first_gait_sequence["onset"],
        first_gait_sequence["onset"] + first_gait_sequence["duration"],
    )
]

# Plot raw acceleration data
for i in range(3):
    ax.plot(
        time_seconds,
        acceleration_data[f"LowerBack_ACCEL_{chr(120 + i)}"],
        color=colors[i],
        label=f"Acc {i + 1}",
    )

# Plot gait onset and duration
plt.axvline(first_gait_sequence["onset"], color="green", linestyle="-")
ax.axvspan(
    first_gait_sequence["onset"],
    first_gait_sequence["onset"] + first_gait_sequence["duration"],
    alpha=0.2,
    color="gray",
    label="Gait duration",
)

# Plot initial contacts within the first gait sequence
for ic_time in ic_within_gait["onset"]:
    ax.axvline(ic_time, color="blue", linestyle="--")

# Customize plot
start_limit = first_gait_sequence["onset"] - 1
end_limit = first_gait_sequence["onset"] + first_gait_sequence["duration"] + 1
ax.set_xlim(start_limit, end_limit)
ax.set_ylim(-1, 1.5)
ax.set_xlabel("Time (seconds)", fontsize=16)
ax.set_ylabel("Acceleration (g)", fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
ax.grid(True, linestyle='--', linewidth=0.5)
ax.legend(fontsize=12)
plt.title("Acceleration Data and Gait Events", fontsize=18)
ax.legend(
    ["Acc 1", "Acc 2", "Acc 3", "Gait onset", "Gait duration", "Initial contacts"],
    fontsize=20,
    loc="upper right",
)
plt.tight_layout()

# Save the figure as a PNG
plt.savefig('figure_1.png', dpi=300, bbox_inches='tight')

# Save the figure as a PDF
plt.switch_backend('agg')
plt.savefig('figure_1.pdf', format='pdf', dpi=300, bbox_inches='tight')

# Show the plot
plt.show()


86 gait sequence(s) detected.


C:\Users\Project\AppData\Local\Temp\ipykernel_14296\3269734965.py:115: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


<Figure size 640x480 with 0 Axes>

Since our primary focus in this study is on comparing the results of the Python and MATLAB-based GSD algorithms, it can be concluded that there is an acceptable agreement among the measurement indices. Specifically, sensitivity ranged from 0.86 to 1, specificity from 0.99 to 1, accuracy from 0.99 to 1, and PPV from 0.89 to 1.
When comparing MATLAB results to the reference data and to Python results, we see no significant differences between the two implementations.

# Summary

The results of the Python-based GSD algorithm are comparable to the MATLAB-based GSD algorithm, not only in healthy adults, but across different clinical cohorts. This is the first step toward the development of a more extensive open-source toolbox for the analysis of human motion data.

# References
